Julien Gauthier

In [5]:
%pip install stable-baselines3[extra] highway-env

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
import stable_baselines3 as sb3
import gymnasium as gym
from stable_baselines3 import HerReplayBuffer, DDPG
from tqdm.notebook import trange

In [5]:
#@title Import helpers for visualization of episodes
import sys
%pip install tensorboardx gym pyvirtualdisplay
%pip install ffmpeg-python
sys.path.insert(0, 'HighwayEnv/scripts')
from highway_env.utils import record_videos, show_videos

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
env = gym.make("parking-v0", render_mode='human')

model = DDPG(
    "MultiInputPolicy",
    env,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(n_sampled_goal=4, goal_selection_strategy='future'),
    verbose=1,
    device='cuda'
)
model.learn(100000)

episodes = 10

env = gym.make('parking-v0', render_mode='rgb_array')
#env = record_videos(env)

for episode in trange ((episodes), desc="Test episodes"):
    obs, info = env.reset()
    done = truncated = False
    while not (done or truncated):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
env.close()

#show_videos()


In [19]:
model.save('pre-trained-model.h5f')


In [5]:
# Create a new environment
env = gym.make("parking-v0", render_mode='human')
# Load the trained model
loaded_model = DDPG.load('pre-trained-DDPG-HER.h5f', env=env)

nb_tests = 10

# Test the trained model
obs, _ = env.reset()
for episode in range(nb_tests):
    obs, info = env.reset()
    done = truncated = False
    while not (done or truncated):
        action, _ = loaded_model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
env.close()


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
